# Ship data analysis

This tutorial is derived from an [upstream MovingPandas example](https://movingpandas.github.io/movingpandas-website/2-analysis-examples/ship-data.html).


```
pip install lonboard movingpandas "geopandas>=1" pyarrow
```


In [1]:
import warnings
from datetime import datetime, timedelta
from os.path import exists
from urllib.request import urlretrieve

import geopandas as gpd
import movingpandas as mpd
import pandas as pd

from lonboard import viz, Map, PathLayer
from lonboard.experimental import TripsLayer
from lonboard.colormap import apply_categorical_cmap
from matplotlib.colors import to_rgba
import pyarrow as pa
# import folium

# import hvplot.pandas
# import matplotlib.pyplot as plt

# import numpy as np
# import shapely as shp
# from geopandas import GeoDataFrame, read_file
# from holoviews import dim, opts
# from shapely.geometry import LineString, Point, Polygon


/Users/kyle/github/developmentseed/lonboard/.venv/lib/python3.11/site-packages/movingpandas/__init__.py:37: UserWarning: Missing optional dependencies. To use the trajectory smoother classes please install Stone Soup (see https://stonesoup.readthedocs.io/en/latest/#installation).
  warnings.warn(e.msg, UserWarning)


## Loading sample AIS data

We'll load the same AIS data as the upstream notebook.


In [2]:
url = "https://raw.githubusercontent.com/movingpandas/movingpandas-examples/refs/heads/main/data/ais.gpkg"
gdf = gpd.read_file(url, use_arrow=True)

Let's see what the data looks like:


In [3]:
gdf.head()

,Timestamp,MMSI,NavStatus,SOG,COG,Name,ShipType,geometry
0,05/07/2017 00:00:03,219632000,Under way using engine,0.0,270.4,None,Undefined,POINT (11.85958 57.68817)
1,05/07/2017 00:00:05,265650970,Under way using engine,0.0,0.5,None,Undefined,POINT (11.84175 57.6615)
2,05/07/2017 00:00:06,265503900,Under way using engine,0.0,0.0,None,Undefined,POINT (11.9065 57.69077)
3,05/07/2017 00:00:14,219632000,Under way using engine,0.0,188.4,None,Undefined,POINT (11.85958 57.68817)
4,05/07/2017 00:00:19,265519650,Under way using engine,0.0,357.2,None,Undefined,POINT (11.87192 57.68233)


And let's plot the raw data on a map:


In [4]:
viz(gdf)

Map(basemap_style=<CartoBasemap.DarkMatter: 'https://basemaps.cartocdn.com/gl/dark-matter-gl-style/style.json'…

As with the upstream example, let's remove any records where the ships aren't moving.


In [5]:
print(f"Original size: {len(gdf)} rows")
gdf = gdf[gdf["SOG"] > 0]
print(f"Reduced to {len(gdf)} rows after removing 0 speed records")


Original size: 84702 rows
Reduced to 33593 rows after removing 0 speed records


## Creating trajectories


In [6]:
gdf["t"] = pd.to_datetime(gdf["Timestamp"], format="%d/%m/%Y %H:%M:%S")
traj_collection = mpd.TrajectoryCollection(gdf, "MMSI", t="t", min_length=100)
print(f"Finished creating {len(traj_collection)} trajectories")

Finished creating 77 trajectories


In [7]:
traj_collection = mpd.MinTimeDeltaGeneralizer(traj_collection).generalize(
    tolerance=timedelta(minutes=1)
)

## Plotting trajectories


In [8]:
shiptype_to_color = {
    "Passenger": "blue",
    "HSC": "green",
    "Tanker": "red",
    "Cargo": "orange",
    "Sailing": "grey",
    "Other": "grey",
    "Tug": "grey",
    "SAR": "grey",
    "Undefined": "grey",
    "Pleasure": "grey",
    "Dredging": "grey",
    "Law enforcement": "grey",
    "Pilot": "grey",
    "Fishing": "grey",
    "Diving": "grey",
    "Spare 2": "grey",
}

In [9]:
ship_types = []
for traj in traj_collection.trajectories:
    unique_vals = traj.df["ShipType"].unique()
    # Apparently this is not always consistent
    # assert len(unique_vals) == 1, "Expected single ship type per trajectory"
    ship_types.append(unique_vals[0])

In [10]:
get_color = apply_categorical_cmap(pa.array(ship_types), shiptype_to_color)

In [12]:
trips_layer = TripsLayer.from_movingpandas(
    traj_collection, get_color=get_color, width_min_pixels=10
)

/Users/kyle/github/developmentseed/lonboard/lonboard/experimental/traits.py:146: UserWarning: Reducing precision of input timestamp data to 's' to fit into available GPU precision.
  warnings.warn(


In [13]:
linestring_layer = PathLayer(
    table=trips_layer.table, get_color=get_color, width_min_pixels=1
)

In [14]:
m = Map(linestring_layer)
m

Map(custom_attribution='', layers=(PathLayer(get_color=arro3.core.ChunkedArray<FixedSizeList(Field { name: "",…

In [35]:
layer.width_min_pixels = 20

In [34]:
m

Map(custom_attribution='', layers=(TripsLayer(get_color=arro3.core.ChunkedArray<FixedSizeList(Field { name: ""…

In [ ]:
traj_collection.plot(
    column="ShipType", column_to_color=shiptype_to_color, linewidth=1, capstyle="round"
)
